In [ ]:
!pip install transformers # I use this to load the pretrained BERT
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
import re
import string
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

VER=2
# TRAIN WITH SUBSET OF 60K
NUM_TRAIN_SAMPLES = 1_024
# PARAMETER EFFICIENT FINE TUNING
# PEFT REQUIRES 1XP100 GPU NOT 2XT4
USE_PEFT = False
# NUMBER OF LAYERS TO FREEZE 
# DEBERTA LARGE HAS TOTAL OF 24 LAYERS
FREEZE_LAYERS = 18
# BOOLEAN TO FREEZE EMBEDDINGS
FREEZE_EMBEDDINGS = True
# LENGTH OF CONTEXT PLUS QUESTION ANSWER
MAX_INPUT = 256
# HUGGING FACE MODEL
MODEL = 'microsoft/deberta-v3-large'

In [ ]:
!pip install -U /kaggle/input/faiss-cpu-173/faiss_cpu-1.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers

In [ ]:
!pip install -U /kaggle/working/sentence-transformers


In [ ]:
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl


In [ ]:
def clean_text(text):
    " Function to clean text and keep only relevant ones"
    
    # Remove Emojis
    emoji_pattern = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticons
                          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                          u"\U0001F680-\U0001F6FF"  # transport & map symbols
                          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          u"\U00002702-\U000027B0"
                          u"\U000024C2-\U0001F251"
                          "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [ ]:
# I'm going to use cross validation to get unbiased performance of the model on the training data

count_vectorizer = feature_extraction.text.CountVectorizer() # Using the common bag of words technique
train_vectors = count_vectorizer.fit_transform(x_train)
baseline_model =  linear_model.LogisticRegression()
f1_scores = model_selection.cross_val_score(baseline_model, train_vectors, y_train, cv=3, scoring="f1")
accuracy_scores = model_selection.cross_val_score(baseline_model, train_vectors, y_train, cv=3, scoring="accuracy")
print(f"Cross Validation Accuracy Scores:", accuracy_scores)
print(f"Cross Validation Accuracy f1_score:", f1_scores)


In [ ]:
baseline_model.fit(train_vectors, y_train)

In [ ]:
# Getting the scores on the test:

test_vectors = count_vectorizer.transform(x_test)
baseline_predict_test = baseline_model.predict(test_vectors)
print("Accuracy:", accuracy_score(baseline_predict_test, y_test))
print("F1_score:", f1_score(baseline_predict_test, y_test))

In [ ]:
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example

tokenized_train_ds = train_ds.map(preprocess, batched=False, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

In [ ]:
# Importing a pretrained BERT model so that I can use it to extract richer features of my dataset:

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 
                                                    'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
# Following datacollator (adapted from https://huggingface.co/docs/transformers/tasks/multiple_choice)
# will dynamically pad our questions at batch-time so we don't have to make every question the length
# of our longest question.
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
# The BERT model works only when all vectors have the same length. But because some tweets are longer than others, I have to making all 
#vectors the same length by adding zeroes to vectors with shorter lengths (this shouldn't affect the feautres of the sentence)


max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) #padding with zeroes
padded.shape

In [ ]:
# These are the masks that hide 15% of the words as I explained in my report
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
# Now, I pass the data to the model to extract the richer features (I will technically take the output of the last layer):

input_ids = torch.tensor(padded) #.to(torch.int64)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
#As I explained in the report, I only take the CLS(its index is 0) vector because it represents the entire sentence

features = last_hidden_states[0][:,0,:].numpy() 
labels = kaggle_train['target']

In [ ]:
# I will use the tokenization script provided by google:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
import tokenization

In [ ]:
def text_encoder(texts, tokenizer, max_len=512):
  # this function will take the text and the tokenizer provided by google and will turn them into the three input types: tokens(words), masks(15% of the words), 
                                                                                                                                            # segments(pairs of sentences)
    all_tokens = [] # bag of words
    all_masks = [] # masks which are words that will be hidden and the model have to predict as part of the training
    all_segments = [] #pairs of sentences that will help the model in learning the sequence
    
    for text in texts:
        text = tokenizer.tokenize(text) # Turning the text into tokens
            
        text = text[:max_len-2] # making sure that all sentences have the same length
        input_sequence = ["[CLS]"] + text + ["[SEP]"] # adding the "CLS":classification and "SEP":sentence separation
                                                      # This tells the model when sentence start and end. This will be the segment input that will train the model
                                                      #on the sequence

        pad_len = max_len - len(input_sequence) # Bert takes as input a padded array
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) #turning the each word into its corresponding ID in the Bert model

        # For Bert to work, all tokens vectors should have the same length, so here I'm adding 0s to vectors with smaller length to ensure
        # that all have the same length. Adding 0 doesn't affect the features because 0 is not an ID for any word in the BERT model
        tokens += [0] * pad_len #
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [ "[CLS] " + example['context'] ] * 5
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first', 
                                  max_length=MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")
# FAISS on WIKI articles

In [ ]:
# encode the prompts
prompt_embeddings = model.encode(trn.prompt.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True).half()
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
search_score, search_index = sentence_index.search(prompt_embeddings, 3)


In [ ]:
def build_model(bert_layer, max_len=512): 
    # The Bert model is not mainly designed for classification, so I will add one layer to do classification

    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids") #the tokens
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask") # the masks (hidden words)
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids") # the pairs of sentences

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids]) #extracting the features
    clf_output = sequence_output[:, 0, :] # the BERT model by default returns a vector that represents each word besides one vector that represents all the sentence 
                                          # This is the vector that has the same index (0) as the "CLS" that I inputed above so I'm building classification with that vector
                                          # and I will ignore the other vectors as they are used for other purposes

    out = Dense(1, activation='sigmoid')(clf_output) # Since I have only two classes, I'm adding one layer with a sigmoid function to do the classification
    
    # Adding everything together:
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out) 
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    # I'm using the same paramters used in the origional paper: Adam's optimizer and binary crossentropy which will work on maximizing the likelihood 
    
    return model

In [ ]:
#%%time

# loading the pretrained model:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
model = AutoModelForMultipleChoice.from_pretrained(MODEL)


In [ ]:
# splitting the data to evaluate the model
x_train, x_test, y_train, y_test = train_test_split(kaggle_train['text'], 
                                                    kaggle_train['target'], 
                                                    test_size=0.2, 
                                                    random_state=1)



In [ ]:
#processing the data using the functions above

train_input = text_encoder(x_train, tokenizer, max_len=160)
test_input = text_encoder(x_test, tokenizer, max_len=160)
train_labels = y_train

In [ ]:
#finally training the model

model = build_model(bert_layer, max_len=160)
model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    batch_size=16)

In [ ]:
y = model.predict(test_input)

In [ ]:
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission_sample.csv')
test_input = text_encoder(test['text'], tokenizer, max_len=160)


test_pred = model.predict(test_input).round().astype(int)

submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission1.csv', index=False)